In [1]:
from sentence_transformers import SentenceTransformer, util
import os
import json
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
from torch.utils.data import Dataset
from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoModelForSeq2SeqLM,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq,
    TrainerCallback
)

torch.manual_seed(101)

In [2]:
# Loading bm-25 rankings

df = pd.read_json("../data/bm_25_ranking.json")
df.head()

,rel_rank,query,doc
0,1,foods and supplements to lower blood sugar,A healthy diet is essential to reversing predi...
1,2,foods and supplements to lower blood sugar,"Ohio State University, researchers saw insulin..."
2,3,foods and supplements to lower blood sugar,Food And Supplements That Lower Blood Sugar Le...
3,4,foods and supplements to lower blood sugar,But studies show that eating certain foods may...
4,5,foods and supplements to lower blood sugar,7 easy tips to reduce insulin and lose belly f...


In [3]:
# Loading and reranking using bert-base

def rerank(query, docs):
    model = SentenceTransformer('sentence-transformers/msmarco-bert-base-dot-v5')
    query_emb = model.encode(query)
    doc_emb = model.encode(docs)
    
    scores = util.dot_score(query_emb, doc_emb)[0].cpu().tolist()
    doc_score_pairs = list(zip(docs, scores))

    doc_score_pairs = sorted(doc_score_pairs, key=lambda x: x[1], reverse=True)
    return doc_score_pairs

In [8]:
# Example re-ranking

query = df["query"].unique()[0]
docs = df[df["query"] == query]["doc"].values
doc_score_pairs = rerank(query, docs)

for idx, (doc, score) in enumerate(doc_score_pairs):
    print(f"{idx}, {doc[:100]}...\n")

0, Food And Supplements That Lower Blood Sugar Levels. Cinnamon: Researchers are finding that cinnamon ...

1, Many natural supplements can affect your blood sugar, including niacin, DHEA, ginkgo biloba, melaton...

2, Artichoke and Celery are also good food for lowering blood sugar quickly and naturally. Spinach, Kal...

3, Barley and oats have the same cholesterol lowering fiber which helps to reduce blood sugar. Foods wi...

4, Fish is a great food to lower blood sugar levels because of its overall nutrition, but remember to p...

5, Cinnamon, people who have diabetes, is commonly used to reduce blood sugar and cholesterol level in ...

6, Cinnamon, people who have diabetes, is commonly used to reduce blood sugar and cholesterol level in ...

7, Lemons are rich in Vitamin C and their acidity helps to lower other foods' glycemic indexes. Oat and...

8, Other foods worth mentioning for lower blood sugar levels are coconut butter, dark chocolate, cinnam...

9, Here are some good exampl